# LangChain의 RAG 파헤치기

![rag-1.png](https://github.com/teddylee777/langchain-kr/blob/main/12-RAG/image/rag-1.png)

![rag-2.png](https://github.com/teddylee777/langchain-kr/blob/main/12-RAG/image/rag-2.png)

### 1. 질문 처리

질문 처리 단계에서는 사용자의 질문을 받아 이를 처리하고, 관련 데이터를 찾는 작업이 이루어집니다. 이를 위해 다음과 같은 구성 요소들이 필요합니다:

- **데이터 소스 연결**: 질문에 대한 답변을 찾기 위해 다양한 텍스트 데이터 소스에 연결해야 합니다. LangChain은 다양한 데이터 소스와의 연결을 간편하게 설정할 수 있도록 돕습니다.
- **데이터 인덱싱 및 검색**: 데이터 소스에서 관련 정보를 효율적으로 찾기 위해, 데이터는 인덱싱되어야 합니다. LangChain은 인덱싱 과정을 자동화하고, 사용자의 질문과 관련된 데이터를 검색하는 데 필요한 도구를 제공합니다.

### 2. 답변 생성

관련 데이터를 찾은 후에는 이를 기반으로 사용자의 질문에 답변을 생성해야 합니다. 이 단계에서는 다음 구성 요소가 중요합니다:

- **답변 생성 모델**: LangChain은 고급 자연어 처리(NLP) 모델을 사용하여 검색된 데이터로부터 답변을 생성할 수 있는 기능을 제공합니다. 이러한 모델은 사용자의 질문과 검색된 데이터를 입력으로 받아, 적절한 답변을 생성합니다.


## 아키텍처

우리는 [Q&A 소개](https://python.langchain.comhttps://python.langchain.com/docs/use_cases/question_answering/)에서 개요한 대로 전형적인 RAG 애플리케이션을 만들 것입니다. 이것은 두 가지 주요 구성 요소를 가지고 있습니다:

- **인덱싱**: 소스에서 데이터를 수집하고 인덱싱하는 파이프라인입니다. _이 작업은 보통 오프라인에서 발생합니다._

- **검색 및 생성**: 실제 RAG 체인으로, 사용자 쿼리를 실행 시간에 받아 인덱스에서 관련 데이터를 검색한 다음, 그 데이터를 모델에 전달합니다.

RAW 데이터에서 답변을 받기까지의 전체 순서는 다음과 같습니다.

### 인덱싱

![](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

1. **로드**: 먼저 데이터를 로드해야 합니다. 이를 위해 [DocumentLoaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)를 사용할 것입니다.
2. **분할**: [Text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)는 큰 `Documents`를 더 작은 청크로 나눕니다. 이는 데이터를 인덱싱하고 모델에 전달하는 데 유용하며, 큰 청크는 검색하기 어렵고 모델의 유한한 컨텍스트 창에 맞지 않습니다.
3. **저장**: 나중에 검색할 수 있도록 분할을 저장하고 인덱싱할 장소가 필요합니다. 이는 종종 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/)와 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) 모델을 사용하여 수행됩니다.

### 검색 및 생성

![](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

1. **검색**: 사용자 입력이 주어지면 [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/)를 사용하여 저장소에서 관련 분할을 검색합니다.
2. **생성**: [ChatModel](https://python.langchain.com/docs/modules/model_io/chat/) / [LLM](https://python.langchain.com/docs/modules/model_io/llms/)은 질문과 검색된 데이터를 포함한 프롬프트를 사용하여 답변을 생성합니다




In [ ]:
!git clone https://github.com/aisecuritynlp/aisec.git
!mv ./aisec/* ./

In [1]:
# setup
!pip install python-dotenv==1.0.1
!pip install langchain==0.1.12
!pip install langchain-community==0.0.28
!pip install langchain-experimental==0.0.54
!pip install langchain-core==0.1.32
!pip install langchain-openai==0.0.8
!pip install langsmith==0.1.27
!pip install langchainhub==0.1.15
!pip install pypdf==4.1.0
!pip install faiss-cpu==1.8.0
!pip install fastembed -q
!pip install sentence_transformers
# !pip install sentence_transformers==2.6.1
!pip install pdfplumber
# !pip install --force-reinstall charset-normalizer==3.1.0 --user
!pip install PyMuPDF
# !pip install unstructured unstructured-inference
!pip install --upgrade chardet
!pip install -U httpx
!pip install chromadb
!pip install rank_bm25
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
unstructured 0.12.6 requires urllib3==1.26.18, but you have urllib3 2.2.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unstructured-inference 0.7.25 requires onnxruntime<1.16, but you have onnxruntime 1.17.1 which is incompatible.
ERROR: Could not find a version that satisfies the requirement sentence_transformers==4.38.2 (from v

## 0) Setting

API KEY 를 설정합니다.

'.env' 파일을 만들고, 그 안에 OpenAI의 API 키를 저장

ex) OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [2]:
# import
import os
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pprint import pprint


In [3]:
# # API 키를 환경변수로 관리하기 위한 설정 파일
# from dotenv import load_dotenv
# import os

# # API 키 정보 로드
# load_dotenv()
# # os.environ['OPENAI_API_KEY'] = 'sk   -   xwLy4kTzJD6qxl8jqJt7T3BlbkFJOJsz3o96g86wElid0CBe   '
# api_key = os.environ.get("OPENAI_API_KEY")
# print('API KEY: %s...'%api_key[:-30])

In [4]:
OPENAI_API_KEY = 'sk   -   proj-d36MFnJ08dMAz6YqCPpUT3BlbkFJbBTv8Jpldn1UZQJUh9x4   '
OPENAI_API_KEY = OPENAI_API_KEY.replace(" ", "")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
api_key = os.environ.get("OPENAI_API_KEY")
print('API KEY: %s...'%api_key[:-30])

API KEY: sk-xwLy4kTzJD6qxl8jqJ...


In [5]:
# check
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

answer = llm.invoke(question)

# 질의
print(f"## 질문: {question}")
print(f"## 답변\n{answer}")

## 질문: 대한민국의 수도는 어디인가요?
## 답변
content='대한민국의 수도는 서울입니다.' response_metadata={'finish_reason': 'stop', 'logprobs': None}


In [6]:
# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/mnews/article/584/0000026285"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(docs)

print('#'*70)
print('## chunking ##')

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(docs)
for tmp in splits:
    print(f'## {tmp}\n')

[Document(page_content="\nEU, 세계 최초 'AI 규제법' 통과…위반시 매출 7%까지 과징금\n\n\n\n\n\n\n유럽연합(EU)이 세계 최초로 인공지능(AI)를 규제하는 법을 통과시켰다. 게티이미지뱅크 제공   유럽연합(EU)이 세계 최초로 인공지능(AI)을 규제하는 법을 통과시켰다.    13일(현지시간) 유럽의회는 프랑스 스트라스부르에서 열린 본회의에서 'AI 법(AI Act)' 최종안을 찬성 523표, 반대 46표로 가결했다. 기권은 49표였다. EU는 이 법안이 빠른 속도로 발전하는 기술의 위험으로부터 시민을 보호하는 동시에 혁신을 촉진할 것이라고 보고 있다.    법안에 따르면 EU는 AI 활용 분야를 총 4단계의 위험 등급으로 나눠 차등 규제한다. 고위험 등급으로 분류되는 의료, 교육을 비롯한 공공 서비스나 선거, 핵심 인프라, 자율주행 등에서는 AI 기술 사용 시 사람이 반드시 감독하도록 하고 위험관리시스템을 구축해야 한다.   또 광범위한 사이버 공격, ‘유해한 선입견’ 전파 등 EU가 시스템적 위험이라고 규정한 사고 발생을 방지하기 위한 조치를 해야 한다. 별도의 정보 공개·고지 의무도 부과된다.   사람과 유사한 수준 또는 그 이상의 지능을 갖춘 AI인 '범용 AI'를 개발하는 기업에 대해서는 기술 개발 과정에서의 ‘투명성 의무’를 부여하기로 했다. 이 조항은 2021년 발의된 초안에는 없었지만 이듬해 챗GPT 등 생성형 AI가 보편화되면서 AI 오남용에 대한 우려가 확산돼 입법 과정에서 추가됐다.   일부 AI 기술 활용은 원천 금지된다. 이른바 개인의 특성·행동과 관련된 데이터로 개별 점수를 매기는 관행인 ‘사회적 점수 평가’(소셜 스코어링)가 대표적이다. AI를 활용한 실시간 원격 생체인식 식별 시스템 사용도 중대 범죄 용의자 수색을 비롯해 예외적인 경우를 제외하곤 사실상 금지된다. 이 밖에 딥페이크 영상이나 이미지는 AI로 만든 조작 콘텐츠라는 점을 표기하도록 했다.   이번에 통과된 법안은 EU 27개국 장관

In [7]:
# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "ai 규제법에 대해 알려줘"
response = rag_chain.invoke(question)

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/mnews/article/584/0000026285
문서의 수: 1
[HUMAN]
ai 규제법에 대해 알려줘

[AI]
유럽연합(EU)은 세계 최초로 인공지능(AI)를 규제하는 법을 통과시켰다. 법안에 따라 AI 활용 분야를 4단계의 위험 등급으로 나눠 차등 규제하며, 위반 시 매출의 1.5%에서 최대 7%에 해당하는 과징금이 부과될 수 있다. 이번에 통과된 법안은 EU 27개국 장관들이 내달 최종 승인하고 오는 6월 EU 관보에 게재된 뒤 발효된다.


## 단계 1: 문서 로드(Load Documents)

- [공식문서 링크 - Document loaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)


### 1.1) 웹페이지

`WebBaseLoader`는 지정된 웹 페이지에서 필요한 부분만을 파싱하기 위해 `bs4.SoupStrainer`를 사용합니다.

[참고]

- `bs4.SoupStrainer` 는 편리하게 웹에서 원하는 요소를 가져올 수 있도록 해줍니다.

(예시)

```python
bs4.SoupStrainer(
    "div",
    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]}, # 클래스 명을 입력
)

bs4.SoupStrainer(
    "article",
    attrs={"id": ["dic_area"]}, # 클래스 명을 입력
)
```


아래의 BBC 뉴스 기사입니다. 영문으로 작성된 기사로 시험해 보고 싶다면, 아래의 주석을 해제하고 실행해 보세요.


In [9]:
# 뉴스기사의 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
from langchain_community.document_loaders import WebBaseLoader
url = 'https://n.news.naver.com/mnews/article/584/0000026285'

loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)
docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs[0].page_content[:500].strip()

문서의 수: 1


"EU, 세계 최초 'AI 규제법' 통과…위반시 매출 7%까지 과징금\n\n\n\n\n\n\n유럽연합(EU)이 세계 최초로 인공지능(AI)를 규제하는 법을 통과시켰다. 게티이미지뱅크 제공   유럽연합(EU)이 세계 최초로 인공지능(AI)을 규제하는 법을 통과시켰다.    13일(현지시간) 유럽의회는 프랑스 스트라스부르에서 열린 본회의에서 'AI 법(AI Act)' 최종안을 찬성 523표, 반대 46표로 가결했다. 기권은 49표였다. EU는 이 법안이 빠른 속도로 발전하는 기술의 위험으로부터 시민을 보호하는 동시에 혁신을 촉진할 것이라고 보고 있다.    법안에 따르면 EU는 AI 활용 분야를 총 4단계의 위험 등급으로 나눠 차등 규제한다. 고위험 등급으로 분류되는 의료, 교육을 비롯한 공공 서비스나 선거, 핵심 인프라, 자율주행 등에서는 AI 기술 사용 시 사람이 반드시 감독하도록 하고 위험관리시스템을 구축해야 한다.   또 광범위한 사이버 공격, ‘유해한 선입견’ 전파 등 EU가 시스템적 위험"

### 1.2) PDF


#### 실습 PDF
- [챗GPT 등 생성형 AI 활용 보안 가이드라인](https://www.ncsc.go.kr:4018/main/cop/bbs/selectBoardArticle.do?bbsId=InstructionGuide_main&nttId=54340&pageIndex=1&searchCnd2=#LINK)
- [국가공공기관 민간클라우드컴퓨팅서비스 도입 가능 목록](https://www.ncsc.go.kr:4018/main/cop/bbs/selectBoardArticle.do?bbsId=InstructionGuide_main&nttId=115751&pageIndex=1&searchCnd2=#LINK)

In [14]:
# load pdf 1) PyPDFLoader
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
file_path_pdf = "data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf"
# file_path_pdf = "data/PDF_3_Karspersky_Lazarus_attacked_cryptocurrency_company_with_LPEClient_and_Gopuram.pdf"

loader = PyPDFLoader(file_path_pdf)

# 페이지 별 문서 로드
docs = loader.load()
print(f"## 문서의 수: {len(docs)} ##")

# 10번째 페이지의 내용 출력
print(f"\n## content ##\n{docs[9*int(len(docs)/10)-1].page_content[:500]}\n")
print('## metadata ##')
pprint(docs[1].metadata)

## 문서의 수: 60 ##

## content ##
4 8챗GPT 등 생성형 AI 활용 보안 가이드라인
[정보 유출 방지를 위한 방안 마련] 
-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 
또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 
정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential  privacy )을 적용할 
수 있다.
•평가 및 테스트 단계
[어뷰징 질문에 대한 답변 거부]
 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 
조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 
등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 
규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 
받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환
하여 

## metadata ##
{'page': 1, 'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf'}


In [15]:
# load pdf 2) PDFPlumberLoader
# 예제 코드에 가장 많이 출현하는 방식
# • 평균적으로 우수(한글 인코딩 처리, 속도, metadata)
# • page 단위로 데이터를 로드: 20 page PDF 파일 로드시: 20개의 문서로 로드
# • metadata
#   • source: 파일명
#   • page: 페이지 번호 표기
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader(file_path_pdf)
# 페이지 별 문서 로드
docs = loader.load()
print(f"## 문서의 수: {len(docs)} ##")

# 10번째 페이지의 내용 출력
print(f"\n## content ##\n{docs[9*int(len(docs)/10)-1].page_content[:500]}\n")
print('## metadata ##')
pprint(docs[1].metadata)

## 문서의 수: 60 ##

## content ##
[정보 유출 방지를 위한 방안 마련]
- 개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보
또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈(noise)를 삽입하여
정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘(differential privacy)을 적용할
수 있다.
•평가 및 테스트 단계
[어뷰징 질문에 대한 답변 거부]
개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세
조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성
등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인
규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를
받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환
하여 훈련 데이터 추출 공격을 예방할 수 있다.
[AI 모델의 출력 결과 제한]
- 

## metadata ##
{'CreationDate': "D:20230626102018+09'00'",
 'Creator': 'Adobe InDesign 15.0 (Windows)',
 'ModDate': "D:20230628095130+09'00'",
 'Producer': 'Adobe PDF Library 15.0',
 'Trapped': 'False',
 'file_path': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf',
 'page': 1,
 'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf',
 'total_pages': 60}


In [16]:
# load pdf 3) fitz
# • 단순하게 모든 Text 를 읽어서 하나의 문자열을 합칠 때 유용
# • 전체 페이지 요약
# • 페이지를 읽는 속도가 가장 빠름
# • 페이지 번호 제공
# • 페이지 번호를 제외한 metadata 미지원
import fitz

docs = fitz.open(file_path_pdf) # open a document
print(f"## 문서의 수: {len(docs)} ##")

list_txt = []
for page in docs:
    list_txt.append(page.get_text())

# 1번째 페이지의 내용 출력
print(f"\n## content ##\n{list_txt[9*int(len(docs)/10)-1][:500]}\n")
print('## metadata ##')
pprint(docs.metadata)

## 문서의 수: 60 ##

## content ##
48
챗GPT 등 생성형 AI 활용 보안 가이드라인
[정보 유출 방지를 위한 방안 마련] 
- 
개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 
또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈(noise)를 삽입하여 
정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘(differential privacy)을 적용할 
수 있다.
•평가 및 테스트 단계
[어뷰징 질문에 대한 답변 거부]
	 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 
조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 
등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 
규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 
받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환
하여 훈련 

## metadata ##
{'author': '',
 'creationDate': "D:20230626102018+09'00'",
 'creator': 'Adobe InDesign 15.0 (Windows)',
 'encryption': None,
 'format': 'PDF 1.6',
 'keywords': '',
 'modDate': "D:20230628095130+09'00'",
 'producer': 'Adobe PDF Library 15.0',
 'subject': '',
 'title': '',
 'trapped': ''}


### 1.3) CSV

CSV 는 페이지 번호 대신 행번호로 데이터를 조회합니다.


In [18]:
# load csv
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 로드
loader = CSVLoader(file_path="./data/csv_example.csv", encoding='UTF-8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n## content ##\n{docs[1].page_content[:500]}\n")
print('## metadata ##')
pprint(docs[0].metadata)

문서의 수: 5

## content ##
제목: AI 윤리 강령
내용: AI 기술 및 시스템 사용 시 윤리적인 원칙을 준수하고 인권을 보호하기 위해 AI 윤리 강령을 제정하였습니다.
링크: https://www.example.com/ai-ethics-code

## metadata ##
{'row': 0, 'source': './data/csv_example.csv'}


### 1.4) TXT 파일


In [19]:
# load txt
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/appendix-keywords.txt", encoding='UTF-8')
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n## content ##\n{docs[0].page_content[:500]}\n")
print('## metadata ##')
pprint(docs[0].metadata)

문서의 수: 1

## content ##
Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding

정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.
예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.
연관키워드: 자연어 처리, 벡터화, 딥러닝

Token

정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.
예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.
연관키워드: 토큰화, 자연어

## metadata ##
{'source': './data/appendix-keywords.txt'}


---


## 단계 2: 문서 분절(Chunking)

### 2.1) CharacterTextSplitter

이것은 가장 간단한 방법입니다. 이 방법은 문자를 기준으로 분할합니다(기본값은 "\n\n") 그리고 청크의 길이를 문자의 수로 측정합니다.

1. 텍스트가 어떻게 분할되는지: 단일 문자 단위
2. 청크 크기가 어떻게 측정되는지: `len` of characters.

시각화 예제: https://chunkviz.up.railway.app/


`CharacterTextSplitter` 클래스는 텍스트를 특정 크기의 청크로 분할하는 기능을 제공합니다.

- `separator` 매개변수는 청크를 구분하는 데 사용되는 문자열을 지정하며, 여기서는 두 개의 개행 문자(`"\n\n"`)를 사용합니다
- `chunk_size`는 각 청크의 최대 길이를 결정합니다
- `chunk_overlap`은 인접한 청크 간에 겹치는 문자의 수를 지정합니다.
- `length_function`은 청크의 길이를 계산하는 데 사용되는 함수를 결정하며, 기본적으로 문자열의 길이를 반환하는 `len` 함수가 사용됩니다.
- `is_separator_regex`는 `separator`가 정규 표현식으로 해석될지 여부를 결정하는 불리언 값입니다.


In [20]:
# load pdf 1) PyPDFLoader
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
file_path_pdf = "data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf"
# file_path_pdf = "data/PDF_2_FireEye_Chinas_Leading_Cyber_Security_Firms_Reveal_Advanced_Foreign_Threat_Groups;_Identify_Little_Known_Actors"
# file_path_pdf = "data/PDF_3_crowdstrike_CSA-240331_Chinese_Threat_Actor_Aims_to_Monetize_Phishing_Through_Payment_Gateway_Fraud.pdf"

loader = PyPDFLoader(file_path_pdf)

# 페이지 별 문서 로드
docs = loader.load()
print(f"## 문서의 수: {len(docs)} ##")

text = docs[9*int(len(docs)/10)-1].page_content
print(text)

## 문서의 수: 60 ##
4 8챗GPT 등 생성형 AI 활용 보안 가이드라인
[정보 유출 방지를 위한 방안 마련] 
-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 
또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 
정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential  privacy )을 적용할 
수 있다.
•평가 및 테스트 단계
[어뷰징 질문에 대한 답변 거부]
 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 
조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 
등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 
규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 
받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환
하여 훈련 데이터 추출 공격을 예방할 수 있다.
[AI 모델의 출력 결과 제한]
- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한
해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면, 
공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 
있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가
하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 
반환하는 형태로 응답을 제한해야 한다.
•배포 및 서비스 단계
[정보 유출 대책 마련] 
- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 
정보로 의심되는 표현이 등장할 경우를 대비하여 적절한 대응체계를 사전에 준비해야 
한다. 각급기관은 국가정보원의 「국가 정보보안 기본지침」 및 개인정보보호위원회의 
「개인정보 유출 대응 매뉴얼」 

In [21]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

이 함수는 `text_splitter` 객체의 `create_documents` 메소드를 사용하여 주어진 텍스트(`state_of_the_union`)를 여러 문서로 분할하고, 그 결과를 `texts` 변수에 저장합니다. 이후 `texts`의 첫 번째 문서를 출력합니다. 이 과정은 텍스트 데이터를 처리하고 분석하기 위한 초기 단계로 볼 수 있으며, 특히 큰 텍스트 데이터를 관리 가능한 크기의 단위로 나누는 데 유용합니다.


In [22]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=10,
                                      separator="\n\n")
text_splitter.split_text(text)

['4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 \n또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 \n정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential  privacy )을 적용할 \n수 있다.\n•평가 및 테스트 단계\n[어뷰징 질문에 대한 답변 거부]\n 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 \n조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 \n등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 \n규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 \n받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환\n하여 훈련 데이터 추출 공격을 예방할 수 있다.\n[AI 모델의 출력 결과 제한]\n- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한\n해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면, \n공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 \n있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가\n하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 \n반환하는 형태로 응답을 제한해야 한다.\n•배포 및 서비스 단계\n[정보 유출 대책 마련] \n- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 \n정보로 의심되는 표현이 등장할 경우를 대비하여 적절한 대응체계를 사전에 준비해야 \n한다. 각급기관은 국가정보원의 「국가 정보보안 기본지침」 및 개인정보보호위원회의 \n「개인정보

In [23]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=10,
                                      separator="\n")
text_splitter.split_text(text)

['4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련]',
 '-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보',
 '또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여',
 '정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential  privacy )을 적용할 \n수 있다.\n•평가 및 테스트 단계\n[어뷰징 질문에 대한 답변 거부]',
 '개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 \n조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성',
 '등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 \n규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를',
 '받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환\n하여 훈련 데이터 추출 공격을 예방할 수 있다.\n[AI 모델의 출력 결과 제한]',
 '- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한\n해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면,',
 '공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 \n있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가',
 '하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 \n반환하는 형태로 응답을 제한해야 한다.\n•배포 및 서비스 단계\n[정보 유출 대책 마련]',
 '- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 \n정보로 의심되는 표현이 등장할 경우를 대비하여 적절한 대응체계를 사전에 준비해야',
 '한다. 각급기관은 국가정보원의 「국가 정보보안 기본

In [24]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=10,
                                      separator=" ")
text_splitter.split_text(text)

['4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n- 개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보',
 '기관 내 주요 정보 \n또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 \n정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘(',
 '정보보호 알고리즘( differential privacy )을 적용할 \n수 있다.\n•평가 및 테스트 단계\n[어뷰징 질문에 대한 답변 거부]\n 개발자는 AI 모델이 의도하지 않은',
 '의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 \n조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 \n등과 같이 사회적으로',
 '같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 \n규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 \n받았을',
 '경우)를 \n받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환\n하여 훈련 데이터 추출 공격을 예방할 수 있다.\n[AI 모델의 출력 결과 제한]\n-',
 '결과 제한]\n- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한\n해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게',
 '결과를 사용자에게 전달한다면, \n공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 \n있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에',
 '각 단어의 신뢰도에 노이즈를 추가\n하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 \n반환하는 형태로 응답을 제한해야 한다.\n•배포 및 서비스',
 '및 서비스 단계\n[정보 유출 대책 마련] \n- 개발자는 AI 모델이 생성한 데이터에 개인정

In [25]:
text_splitter = CharacterTextSplitter(chunk_size=100,
                                      chunk_overlap=0,
                                      separator=" ")
text_splitter.split_text(text)

['4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n- 개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보',
 '또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 \n정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential',
 'privacy )을 적용할 \n수 있다.\n•평가 및 테스트 단계\n[어뷰징 질문에 대한 답변 거부]\n 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한',
 '미세 \n조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 \n등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나,',
 '윤리적인 \n규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 \n받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을',
 '반환\n하여 훈련 데이터 추출 공격을 예방할 수 있다.\n[AI 모델의 출력 결과 제한]\n- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한\n해야',
 '한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면, \n공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 \n있다. 이를',
 '방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가\n하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 \n반환하는 형태로 응답을',
 '제한해야 한다.\n•배포 및 서비스 단계\n[정보 유출 대책 마련] \n- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 \n정보로 의심되는 표현이 등장할',
 '경우를 대비하여 적절한 대응체계를 사전에 준비해야 \n한다. 각급기관은 국가정보원의 「국가

In [26]:
text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=20,
                                      separator=" ")
# text 파일을 청크로 나누어줍니다.
split_docs = text_splitter.split_text(text)
print('chunk 개수: %d'%len(split_docs))
split_docs

chunk 개수: 6


['4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n- 개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 \n또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 \n정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential',
 '알고리즘( differential privacy )을 적용할 \n수 있다.\n•평가 및 테스트 단계\n[어뷰징 질문에 대한 답변 거부]\n 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 \n조정 방식을 고려해야 한다. 예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 \n등과 같이 사회적으로 논란이나 거부감을 일으킬',
 '사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 \n규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 \n받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환\n하여 훈련 데이터 추출 공격을 예방할 수 있다.\n[AI 모델의 출력 결과 제한]\n- 개발자는 입력에 대한 AI',
 '제한]\n- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한\n해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면, \n공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 \n있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가\n하여 교란하거나,',
 '노이즈를 추가\n하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 \n반환하는 형태로 응답을 제한해야 한다.\n•배포 및 서비스 단계\n[정보 유출 대책 마련] \n- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 \n정보로 의

### 2.2) Semantic Similarity

의미적 유사성을 기준으로 텍스트를 분할합니다.

출처: [Greg Kamradt’s Notebook](https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/5_Levels_Of_Text_Splitting.ipynb)

높은 수준(high level)에서 문장으로 분할한 다음 3개 문장으로 그룹화한 다음 임베딩 공간에서 유사한 문장을 병합하는 방식입니다.


In [29]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# SemanticChunker 를 생성합니다.
semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), add_start_index=True)

# text 파일을 청크로 나누어줍니다.
split_docs = semantic_text_splitter.split_text(text)
print('chunk 개수: %d'%len(split_docs))

for sent in split_docs:
    print(sent)
    print("===" * 20)

chunk 개수: 2
4 8챗GPT 등 생성형 AI 활용 보안 가이드라인
[정보 유출 방지를 위한 방안 마련] 
-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 
또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여 
정보 노출의 위험도를 낮춰주기 위한 정보보호 알고리즘( differential  privacy )을 적용할 
수 있다. •평가 및 테스트 단계
[어뷰징 질문에 대한 답변 거부]
 개발자는 AI 모델이 의도하지 않은 질문에 대한 답변을 거부할 수 있도록 신중한 미세 
조정 방식을 고려해야 한다.
예를 들어, 내부 정보의 유출 유도 또는 혐오 발언, 선정성 
등과 같이 사회적으로 논란이나 거부감을 일으킬 수 있는 답변은 거부하거나, 윤리적인 
규범에 따라 조정되어야 한다. 또한 빈 프롬프트(입력이 공백 문자열로만 이루어진 경우)를 
받았을 때는 메시지가 끊긴 것으로 간주하고 답변을 다시 확인해달라는 응답을 반환
하여 훈련 데이터 추출 공격을 예방할 수 있다. [AI 모델의 출력 결과 제한]
- 개발자는 입력에 대한 AI 모델의 모든 출력을 사용자에게 그대로 전달하는 것을 제한
해야 한다. 만약 입력에 대한 AI 모델의 모든 출력 결과를 사용자에게 전달한다면, 
공격자는 이러한 쿼리를 반복하여 수행함으로써 비공개 모델의 복제본을 생성할 수 
있다. 이를 방지하기 위해 사용자에게 반환하는 각 단어의 신뢰도에 노이즈를 추가
하여 교란하거나, 사용자가 가장 선호할 것으로 예상되는 단일 또는 일정 개수의 답안을 
반환하는 형태로 응답을 제한해야 한다. •배포 및 서비스 단계
[정보 유출 대책 마련] 
- 개발자는 AI 모델이 생성한 데이터에 개인정보 또는 비공개 업무자료 등 민감한 
정보로 의심되는 표현이 등장할 경우를 대비하여 적절한 대응체계를 사전에 준비해야 
한다. 각급기관은 국가정보원의 「국가 정보보안 기본지침」 및 개인정보보호위원회의 
「개인정보 유출 대응 매뉴얼」 등을 참

## 3 단계: 임베딩

참고: https://python.langchain.com/docs/integrations/text_embedding


### 3.1) 유료 과금 임베딩(OpenAI)


In [30]:
text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=15,
                                      separator="\n")
# text 파일을 청크로 나누어줍니다.
text_splitter.split_text(text)

# document를 청크로 나누어줍니다.
split_docs = text_splitter.split_documents(docs)
print('chunk 개수: %d'%len(split_docs))
split_docs[:2]

chunk 개수: 228


[Document(page_content='챗GPT 등 \n생성형 AI 활용  \n보안 가이드라인챗GPT 등 \n생성형 AI 활용보안 가이드라인', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 0}),
 Document(page_content='챗GPT 등 \n생성형 AI 활용  \n보안 가이드라인', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 2})]

In [31]:
# vector db 생성
from langchain_community.vectorstores import FAISS, Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.

# embedder_model = "text-embedding-3-small"
# embedder_model = "text-embedding-3-large"
embedder_model = "text-embedding-ada-002"

vectorstore = FAISS.from_documents(documents=split_docs,
                                   embedding=OpenAIEmbeddings(model=embedder_model))

다음은 `OpenAI` 의 지원되는 Embedding 모델들의 목록입니다.

- 기본 값은 `text-embeding-ada-002` 입니다.


| MODEL                  | ROUGH PAGES PER DOLLAR | EXAMPLE PERFORMANCE ON MTEB EVAL |
| ---------------------- | ---------------------- | -------------------------------- |
| text-embedding-3-small | 62,500                 | 62.3%                            |
| text-embedding-3-large | 9,615                  | 64.6%                            |
| text-embedding-ada-002 | 12,500                 | 61.0%                            |


### 3.2) 무료 Open Source 기반 임베딩


In [35]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_docs,
                                   embedding=HuggingFaceBgeEmbeddings(
                                       model_name = "BAAI/bge-small-en",  # "BAAI/bge-large-en"
                                       model_kwargs = {'device': 'cpu'},
                                       encode_kwargs = {'normalize_embeddings': True}))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 4단계: 벡터스토어 생성(Create Vectorstore)


In [38]:
# FAISS DB 적용
from langchain_community.vectorstores import FAISS
# from langchain_community.vectorstores import Chroma

vectorstore = FAISS.from_documents(documents=split_docs,
                                   embedding=OpenAIEmbeddings(model=embedder_model))

# Chroma DB 적용
# vectorstore = Chroma.from_documents(documents=split_docs,
#                                    embedding=OpenAIEmbeddings(model=embedder_model))

vectorstore

## 5단계: Retriever 생성

리트리버는 구조화되지 않은 쿼리가 주어지면 문서를 반환하는 인터페이스입니다.

리트리버는 문서를 저장할 필요 없이 문서를 반환(또는 검색)하기만 합니다.

- [공식 도큐먼트](https://python.langchain.com/docs/modules/data_connection/retrievers/)

생성된 VectorStore 에 `as_retriver()` 로 가져와서 **Retriever** 를 생성합니다.


### 5.1) 유사도 기반 검색

- 기본값은 코사인 유사도인 `similarity` 가 적용되어 있습니다.


In [39]:
query = "생성형 AI를 사용할 때 정보 유출 방지를 위해 어떤것을 고려해야해?"

retriever = vectorstore.as_retriever(search_type="similarity")
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(page_content='사용하게 되면, 정보 노출, 데이터 추론 등을 통해 훈련 데이터의 기밀성이 침해될 뿐 \n아니라, 민감한 개인정보가 노출될 수 있다. \n•모델 학습 단계\n[AI 모델 성능 고려한 기밀 정보 유출 차단 전략 수립] \n-  개발자는 AI 모델의 성능과 기밀 정보의 유출 가능성에 대한 상충 관계를 적절히 조절', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 52}), Document(page_content='4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 \n또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 53}), Document(page_content='사용자들은 AI 모델 생성기반 공격에 대해 비판적으로 접근하고, 편견이나 허위 \n정보를 퍼뜨리는 목적으로 사용되는 콘텐츠를 구별할 수 있어야 한다. 또한, 다양한 \n도구를 사용하고 보안 전문가와 인공지능 개발자들이 함께 시스템을 구축하여 \n이러한 공격에 대응해야 한다.\n인공지능 생성 콘텐츠', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 42}), Document(page_content='정보뿐만 아니라 다양한 형태의 데이터를 악용하여 유해한 행동을 수행할 위험이 있다. \n예를 들어, 모델이 악의적인 이미지를 생성하거나 음성 인증 시스템을 속일 수 있는  \n음성을 생성하는 등의 악용 가능성이 있다.AI 모델 \n적용 범위 확장\n안전성 확인 \

`similarity_score_threshold` 는 유사도 기반 검색에서 `score_threshold` 이상인 결과만 반환합니다.


In [40]:
query = "생성형 AI를 사용할 때 정보 유출 방지를 위해 어떤것을 고려해야해?"

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(page_content='사용하게 되면, 정보 노출, 데이터 추론 등을 통해 훈련 데이터의 기밀성이 침해될 뿐 \n아니라, 민감한 개인정보가 노출될 수 있다. \n•모델 학습 단계\n[AI 모델 성능 고려한 기밀 정보 유출 차단 전략 수립] \n-  개발자는 AI 모델의 성능과 기밀 정보의 유출 가능성에 대한 상충 관계를 적절히 조절', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 52}), Document(page_content='4 8챗GPT 등 생성형 AI 활용 보안 가이드라인\n[정보 유출 방지를 위한 방안 마련] \n-  개발자는 AI 모델의 학습 데이터에 의도하지 않게 포함될 수 있는 기관 내 주요 정보 \n또는 개인정보를 보호하기 위해, 학습 과정에서 임의의 노이즈( noise )를 삽입하여', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 53}), Document(page_content='사용자들은 AI 모델 생성기반 공격에 대해 비판적으로 접근하고, 편견이나 허위 \n정보를 퍼뜨리는 목적으로 사용되는 콘텐츠를 구별할 수 있어야 한다. 또한, 다양한 \n도구를 사용하고 보안 전문가와 인공지능 개발자들이 함께 시스템을 구축하여 \n이러한 공격에 대응해야 한다.\n인공지능 생성 콘텐츠', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 42}), Document(page_content='정보뿐만 아니라 다양한 형태의 데이터를 악용하여 유해한 행동을 수행할 위험이 있다. \n예를 들어, 모델이 악의적인 이미지를 생성하거나 음성 인증 시스템을 속일 수 있는  \n음성을 생성하는 등의 악용 가능성이 있다.AI 모델 \n적용 범위 확장\n안전성 확인 \

`maximum marginal search result` 를 사용하여 검색합니다.


In [41]:
query = "생성형 AI를 사용할 때 정보 유출 방지를 위해 어떤것을 고려해야해?"

retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
search_result = retriever.get_relevant_documents(query)
print(search_result)

[Document(page_content='사용하게 되면, 정보 노출, 데이터 추론 등을 통해 훈련 데이터의 기밀성이 침해될 뿐 \n아니라, 민감한 개인정보가 노출될 수 있다. \n•모델 학습 단계\n[AI 모델 성능 고려한 기밀 정보 유출 차단 전략 수립] \n-  개발자는 AI 모델의 성능과 기밀 정보의 유출 가능성에 대한 상충 관계를 적절히 조절', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 52}), Document(page_content='2 9챗GPT 등 생성형 AI 활용 보안 가이드라인2   서비스 답변의 유해성은 어떻게 피할 수 있는가?\n2   사용자는 서비스가 생성한 결과물을 항상 주의 깊게 검토하고, 부적절한 내용이나 \n유해한 요소가 포함되어 있는지 확인해야 한다. 또한, 관련 규정을 준수하고 \n있는지에 대해서도 주의를 기울여야 한다.', metadata={'source': 'data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf', 'page': 34})]


### 5.2) 다양한 쿼리 생성


In [45]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

query = "생성형 AI를 사용할 때 정보 유출 방지를 위해 어떤것을 고려해야해?"

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

unique_docs = retriever_from_llm.get_relevant_documents(query=query)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. AI 생성형 모델을 사용할 때 정보 유출을 방지하기 위한 고려 사항은 무엇인가요?', '2. 정보 유출 방지를 위해 AI 생성형 기술을 적용할 때 고려해야 할 사항은 무엇인가요?', '3. AI 생성형을 활용할 때 정보 보안을 강화하기 위해 고려해야 할 사항은 무엇인가요?']


8

### 5.3) Ensemble Retriever


In [48]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

doc_list = [
    "난 오늘 많이 먹어서 배가 정말 부르다",
    "떠나는 저 배가 오늘 마지막 배인가요?",
    "내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

faiss_vectorstore = FAISS.from_texts(doc_list, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

def pretty_print(docs):
    for i, doc in enumerate(docs):
        print(f"[{i+1}] {doc.page_content}")

In [49]:
sample_query = "나 요즘 배에 정말 살이 많이 쪘어..."
print(f"[Query]\n{sample_query}\n")

relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
나 요즘 배에 정말 살이 많이 쪘어...

[BM25 Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[3] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.


In [50]:
sample_query = "바다 위에 떠다니는 배들이 많다"
print(f"[Query]\n{sample_query}\n")

relevant_docs = bm25_retriever.get_relevant_documents(sample_query)
print("[BM25 Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = faiss_retriever.get_relevant_documents(sample_query)
print("[FAISS Retriever]")
pretty_print(relevant_docs)
print("===" * 20)
relevant_docs = ensemble_retriever.get_relevant_documents(sample_query)
print("[Ensemble Retriever]")
pretty_print(relevant_docs)

[Query]
바다 위에 떠다니는 배들이 많다

[BM25 Retriever]
[1] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[FAISS Retriever]
[1] 난 오늘 많이 먹어서 배가 정말 부르다
[2] 떠나는 저 배가 오늘 마지막 배인가요?
[Ensemble Retriever]
[1] 떠나는 저 배가 오늘 마지막 배인가요?
[2] 난 오늘 많이 먹어서 배가 정말 부르다
[3] 내가 제일 좋아하는 과일들은 배, 사과, 키워, 수박 입니다.


## 6단계: 언어모델 생성(Create LLM)


OpenAI 모델 중 하나를 선택합니다.

- `gpt-3.5-turbo` : OpenAI의 GPT-3.5-turbo 모델
- `gpt-4-turbo-preview` : OpenAI의 GPT-4-turbo-preview 모델

자세한 비용 체계는 [OpenAI API 모델 리스트 / 요금표](https://teddylee777.github.io/openai/openai-models/)에서 확인할 수 있습니다.


In [51]:
# openai gpt3.5
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

with get_openai_callback() as cb:
    result = model.invoke("AI를 활용한 보안기술과, 보안기술을 위한 AI중 어느것이 중요해?")

print(result)
print(cb)

content='AI를 활용한 보안기술과 보안기술을 위한 AI는 둘 다 중요합니다. \n\nAI를 활용한 보안기술은 보안 전문가들이 AI 기술을 활용하여 보안 위협을 탐지하고 대응하는 데 도움을 줄 수 있습니다. AI는 대량의 데이터를 분석하고 패턴을 식별하는 데 탁월한 성능을 보이기 때문에 보안 시스템을 보다 효율적으로 운영할 수 있습니다.\n\n반대로, 보안기술을 위한 AI는 AI 기술을 개발하고 향상시키는 데 중요합니다. AI를 활용한 보안기술이 효과적으로 작동하기 위해서는 AI 기술 자체가 안전하고 신뢰할 수 있어야 합니다. 따라서 AI를 개발하고 운영하는 과정에서 보안 측면을 고려하는 것이 중요합니다.\n\n결국, 두 가지 모두가 상호 보완적이며 중요하다고 볼 수 있습니다. 보안을 강화하고 향상시키기 위해서는 AI를 활용한 보안기술과 보안기술을 위한 AI를 함께 고려해야 합니다.' response_metadata={'finish_reason': 'stop', 'logprobs': None}
Tokens Used: 432
	Prompt Tokens: 46
	Completion Tokens: 386
Successful Requests: 1
Total Cost (USD): $0.000841


In [52]:
# openai gpt4, 27배 비용
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

model = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")

with get_openai_callback() as cb:
    result = model.invoke("AI를 활용한 보안기술과, 보안기술을 위한 AI중 어느것이 중요해?")

print(result)
print(cb)

content='AI를 활용한 보안기술과 보안기술을 위한 AI 모두 중요합니다. 이 두 분야는 상호 보완적인 관계에 있으며, 각각의 중요성은 그 적용 분야와 목적에 따라 달라질 수 있습니다.\n\n1. **AI를 활용한 보안기술**: 이는 사이버 보안, 비디오 감시, 사기 탐지, 네트워크 보안 등 다양한 분야에서 활용됩니다. AI 기술을 통해 대규모 데이터를 실시간으로 분석하고, 비정상적인 행동이나 패턴을 식별하여 보안 위협을 사전에 감지하고 대응할 수 있습니다. 예를 들어, 머신러닝 알고리즘은 이메일에서 스팸이나 피싱 시도를 필터링하거나, 네트워크 트래픽을 분석하여 해킹 시도를 탐지하는 데 사용될 수 있습니다.\n\n2. **보안기술을 위한 AI**: 이는 AI 시스템 자체의 보안을 강화하는 데 중점을 둡니다. AI 시스템은 데이터 조작, 악의적인 입력, 모델 도용 등 다양한 보안 위협에 취약할 수 있습니다. 따라서, AI 모델의 보안을 강화하고, 데이터의 무결성을 보장하며, AI 시스템을 안전하게 유지하기 위한 기술이 필요합니다. 예를 들어, 적대적 공격에 대응하기 위해 AI 모델을 견고하게 만드는 기술이나, AI 시스템의 결정 과정을 투명하게 만들어 신뢰성을 높이는 기술 등이 있습니다.\n\n결론적으로, AI를 활용한 보안기술은 사이버 공간과 실제 세계에서의 보안 위협으로부터 우리를 보호하는 데 필수적입니다. 반면, 보안기술을 위한 AI는 AI 시스템 자체가 안전하고 신뢰할 수 있도록 보장하는 데 중요합니다. 이 두 분야는 서로 의존적이며, 보안의 전반적인 효과를 극대화하기 위해 함께 발전해 나가야 합니다.' response_metadata={'finish_reason': 'stop', 'logprobs': None}
Tokens Used: 743
	Prompt Tokens: 46
	Completion Tokens: 697
Successful Requests: 1
Total Cost (USD): $0.021369999999999997


## 7단계: RAG

In [53]:
# RAG 전체 코드 2
# import
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from pprint import pprint

In [54]:
##############################################################################
## 1) 분석할 파일 로드
# PDF 파일 로드. 파일의 경로 입력
file_path_pdf = "data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf"
loader = PyPDFLoader(file_path=file_path_pdf)

# 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_docs = loader.load_and_split(text_splitter=text_splitter)

##############################################################################
## 2) 파일 벡터DB 생성
top_k = 2  # 유사도 높은 K 개의 문서를 검색합니다.

# (Sparse) bm25 retriever 생성
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k = top_k

# (Dense) FAISS retriever 생성
faiss_vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": top_k})

# ensemble retriever(Sparse + Dense)
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [55]:
##############################################################################
## 3) LLM 모델 생성
llm_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

##############################################################################
## 4) 프롬프트 생성
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# system_template="""Use the following pieces of context to answer the users question shortly.
# Given the following summaries of a long document and a question, create a final answer with references ("SOURCES"), use "SOURCES" in capital letters regardless of the number of sources.
# If you don't know the answer, just say that "I don't know", don't try to make up an answer.
# ----------------
# {summaries}

# You MUST answer in Korean and in Markdown format. Three bullet point."""


system_template="""
주어진 문서들을 기반으로 question에 대해 답변해줘.
모르면 꼭 "I don't know" 라고 답변해.
----------------
{summaries}

한국말로 답변해.
4줄로 요약해서 bullet point로 정리해서 답변해.
"""

# system_template="""
# 주어진 문서들을 기반으로 question에 대해 답변해줘.
# 모르면 꼭 "I don't know" 라고 답변해.
# ----------------
# {summaries}

# 한국말로 답변해.
# 중학생에게 설명해주듯이 친절하고 차분하며 자세히 알려줘
# """

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]

prompt = ChatPromptTemplate.from_messages(messages)

##############################################################################
## 5) RAG 모델 생성
RAG_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_model,
    chain_type="stuff",
    retriever = ensemble_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [56]:
# test 1)
question = "생성형AI의 보안대책에 대해 알려줘"
answer = RAG_chain.invoke(question)

print(f"## question ##\n{answer['question'].strip()}\n")
print(f"## answer ##\n{answer['answer'].strip()}\n")
print(f"## source ##")
for doc in answer['source_documents']:
    print('#'*80)
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))

## question ##
생성형AI의 보안대책에 대해 알려줘

## answer ##
- 사회적 혼란 조장 및 고위험 의사 결정 방지
- 잘못된 의사 결정 유도 방지
- 인공지능 생성 콘텐츠 위험 인지 강화
- AI 모델 테스트 및 시스템 통합 전략 수립 필요

## source ##
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 5
내용 : 4. 생성형 인공지능 기반 정보화사업 구축 방안 및 보안 대책 	  39 4.1. 개요 	  39 4.2. 구축 유형에 따른 생성형 AI 기술 도입시 고려사항 	  40 4.3. 
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 17
내용 : 1 2챗GPT 등 생성형 AI 활용 보안 가이드라인 •대표 보안 위협 1) 사회적 혼란 조장  AI 모델의 악용 혹은 환각 현상 등에 따라 잘못된 결과물은 사회적 혼란을 조장하거나
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 43
내용 : 3 8챗GPT 등 생성형 AI 활용 보안 가이드라인 인공지능 생성 콘텐츠  위험 인지 »사실 여부 확인 및 객관적 분석을 통한 비판적 콘텐츠 수용  » 감정적 자극 유도 및 소셜 
################################################################################
파일 : data

In [57]:
# test 2)
question = "생성형AI의 종류에 대해 알려줘"
answer = RAG_chain.invoke(question)

print(f"## question ##\n{answer['question'].strip()}\n")
print(f"## answer ##\n{answer['answer'].strip()}\n")
print(f"## source ##")
for doc in answer['source_documents']:
    print('#'*80)
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))

## question ##
생성형AI의 종류에 대해 알려줘

## answer ##
- 생성형 인공지능 기술은 대량의 데이터를 학습하여 새로운 데이터를 생성하는 기술이다.
- 대규모 언어모델은 수백억 개 이상의 파라미터를 포함하며, 언어 패턴과 의미를 학습하여 다양한 추론 작업에 우수한 성능을 보유한다.
- 주요 생성형 대규모 언어모델로는 GPT-3, GShard, LaMDA, OPT 등이 있다.
- 대부분의 대규모 언어모델은 API 등을 통해 제한적으로만 접근 가능하다.

## source ##
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 5
내용 : 4. 생성형 인공지능 기반 정보화사업 구축 방안 및 보안 대책 	  39 4.1. 개요 	  39 4.2. 구축 유형에 따른 생성형 AI 기술 도입시 고려사항 	  40 4.3. 
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 17
내용 : 1 2챗GPT 등 생성형 AI 활용 보안 가이드라인 •대표 보안 위협 1) 사회적 혼란 조장  AI 모델의 악용 혹은 환각 현상 등에 따라 잘못된 결과물은 사회적 혼란을 조장하거나
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 51
내용 : 4 6챗GPT 등 생성형 AI 활용 보안 가이드라인 [테스트 전략]  -  AI 모델 도입 전에는 반드시 해당 모델 결과에 대한

In [58]:
# test 3)
question = "업무 효율성을 위해 생성형AI에 회사 보안정보를 입력해도 되?"
answer = RAG_chain.invoke(question)

print(f"## question ##\n{answer['question'].strip()}\n")
print(f"## answer ##\n{answer['answer'].strip()}\n")
print(f"## source ##")
for doc in answer['source_documents']:
    print('#'*80)
    print('파일 : ' + doc.metadata['source'])
    print('페이지 : ' + str(doc.metadata['page']))
    print('내용 : ' + doc.page_content[0:100].replace('\n', ' '))

## question ##
업무 효율성을 위해 생성형AI에 회사 보안정보를 입력해도 되?

## answer ##
아니요, 회사의 보안 정보는 AI 모델에 입력하면 안 됩니다. 보안 책임자는 기밀 정보를 입력하지 말라고 권고하며, 개인 및 민감 정보가 노출될 위험이 있습니다.

- AI 모델에 회사의 기밀 정보를 입력하지 말 것
- 개인정보 및 민감 정보는 AI 모델에 입력하지 않도록 주의
- 데이터베이스 해킹 및 회원 추론 공격에 대비하여 보안을 강화
- 대화 기록 유출로 인한 개인 정보 노출을 방지하기 위한 조치 필요

## source ##
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 23
내용 : 1 8챗GPT 등 생성형 AI 활용 보안 가이드라인 사용자는 AI 모델을 사용할 때 기관의 기밀 정보를 입력하지 않아야 하며, 기관 정보   보안 책임자는 이러한 정보 유출 방지를
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 14
내용 : 9챗GPT 등 생성형 AI 활용 보안 가이드라인- 그러나 이후 4월 28일, OpenAI 가 DPA의 요구사항을 이행함에 따라 이탈리아 정부는  챗GPT의 접속 차단을 해제하였다.
################################################################################
파일 : data/PDF_1_챗GPT 등 생성형 AI 활용 보안 가이드라인(2023.6).pdf
페이지 : 30
내용 : 2 5챗GPT 등 생성형 AI 활용 보안 가이드라인3. 안전한 생성형 인공지

## 8단계: Gradio UI 만들기

In [59]:
# functions
# RAG 전체 코드 2
# import
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from pprint import pprint
import gradio as gr

# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
import os

# API 키 정보 로드
load_dotenv()
# os.environ['OPENAI_API_KEY'] = 'sk- xwLy4kTzJD6qxl8jqJt7T3BlbkFJOJsz3o96g86wElid0CBe'
api_key = os.environ.get("OPENAI_API_KEY")
print('API KEY: %s...'%api_key[:-30])


def respond(message, chat_history):  # 채팅봇의 응답을 처리하는 함수를 정의합니다.

    result = RAG_chain.invoke(message)

    bot_message = result['answer']

    tmp_string = '\n' + '#'*70 + '\n' + '[source]' + '\n'
    bot_message += tmp_string

    for i, doc in enumerate(result['source_documents']):
        bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + '(' + str(doc.metadata['page']) + ') \n'

    chat_history.append((message, bot_message))  # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가합니다.

    return "", chat_history  # 수정된 채팅 기록을 반환합니다.


def change_RAG_setting(file_path_pdf, embedding_model, LLM_model, instruction):
    print('버튼 실행')
    # 1) file
    loader = PyPDFLoader(file_path=file_path_pdf)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    split_docs = loader.load_and_split(text_splitter=text_splitter)

#     print(split_docs)



    ##############################################################################
    ## 2) 파일 벡터DB 생성
    top_k = 2  # 유사도 높은 K 개의 문서를 검색합니다.

    # (Sparse) bm25 retriever 생성
    bm25_retriever = BM25Retriever.from_documents(split_docs)
    bm25_retriever.k = top_k

    # 2) embedding_model
    if embedding_model.startswith("[무료]"):
        embedding_model = embedding_model.replace("[무료]", "", 1)
        faiss_vectorstore = FAISS.from_documents(documents=split_docs,
                                           embedding=HuggingFaceBgeEmbeddings(
                                               model_name = embedding_model, #"BAAI/bge-small-en",  # "BAAI/bge-large-en"
                                               model_kwargs = {'device': 'cpu'},
                                               encode_kwargs = {'normalize_embeddings': True}))
    else:
        # embedder_model = "text-embedding-3-small"
        # embedder_model = "text-embedding-3-large"
        # embedder_model = "text-embedding-ada-002"
        embedding_model = embedding_model.replace("[유료]OpenAIEmbeddings_", "", 1)
        faiss_vectorstore = FAISS.from_documents(documents=split_docs,
                                           embedding=OpenAIEmbeddings(model=embedding_model))

    faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": top_k})

    # ensemble retriever(Sparse + Dense)
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
    )
    print(ensemble_retriever)

    ##############################################################################
    ## 3) LLM 모델 생성
    llm_model = ChatOpenAI(model_name=LLM_model, temperature=0)


    ##############################################################################
    ## 4) 프롬프트 생성
    messages = [
        SystemMessagePromptTemplate.from_template(instruction),
        HumanMessagePromptTemplate.from_template("{question}")
    ]

    prompt = ChatPromptTemplate.from_messages(messages)

    ##############################################################################
    ## 5) RAG 모델 생성
    global RAG_chain

    RAG_chain = RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm_model,
        chain_type="stuff",
        retriever = ensemble_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

    print('### 준비 완료!! ###')
    return



system_template="""
주어진 문서들을 기반으로 question에 대해 답변해줘.
모르면 꼭 "I don't know" 라고 답변해.
----------------
{summaries}

한국말로 답변해.
4줄로 요약해서 bullet point로 정리해서 답변해.
"""

API KEY: sk-xwLy4kTzJD6qxl8jqJ...


In [ ]:
# ui
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

global RAG_chain

with gr.Blocks(
        gr.themes.Soft(primary_hue=gr.themes.colors.slate,
                       secondary_hue=gr.themes.colors.purple)
) as demo:  # gr.Blocks()를 사용하여 인터페이스를 생성합니다.
    gr.Markdown('''# Langchain을 이용한 RAG 이론 및 실전구축 실습 \n

                ''')
    with gr.Row():
        with gr.Column(scale=0.5, variant='panel'):
            gr.Markdown("## 1) Upload Document & Select the Embedding Model")

            # 파일 업로드
            file_path_pdf = gr.File(type="filepath")

            with gr.Row(equal_height=True):
                # LLM 모델 선택
                with gr.Column(scale=0.5, variant='panel'):
                    LLM_model = gr.Dropdown(choices=[
                        "gpt-3.5-turbo",
                        "gpt-4-turbo-preview",
                    ],
                                            value="gpt-3.5-turbo",
                                            label="2) Select the LLM model",
                                            interactive=True)

                # 임베딩 모델 선택
                with gr.Column(scale=0.5, variant='panel'):
                    embedding_model = gr.Dropdown(
                        choices=[
                            "[유료]OpenAIEmbeddings_text-embedding-ada-002",
                            "[유료]OpenAIEmbeddings_text-embedding-3-small",
                            "[유료]OpenAIEmbeddings_text-embedding-3-large",
                            "[무료]BAAI/bge-small-en",
                            "[무료]BAAI/bge-large-en",
                        ],
                        value="[유료]OpenAIEmbeddings_text-embedding-ada-002",
                        label="3) Select the embedding model",
                        interactive=True)

        with gr.Column(scale=0.5, variant='panel'):
            gr.Markdown("## 4) Setting Prompt")
            # 시스템 프롬프트
            instruction = gr.Textbox(label="System instruction",
                                     lines=3,
                                     value=system_template,
                                    interactive=True)

    # 준비
    start_btn = gr.Button('Prepare RAG (10초 대기)',
                          variant='primary',
                          scale=1)

    start_btn.click(change_RAG_setting,
                    inputs=[file_path_pdf, embedding_model, LLM_model, instruction])

    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(
        label="입력",
        value='생성형AI의 보안대책에 대해 알려줘')  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    msg.submit(respond, [msg, chatbot],
               [msg, chatbot])  # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    clear.click(lambda: None, None, chatbot,
                queue=False)  # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.

demo.launch(
    debug=True,
    share=True
)  # 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.

/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://83745f8e6aec98e0c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


버튼 실행
retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7a053c695660>, k=2), VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a053cbf4640>, search_kwargs={'k': 2})] weights=[0.5, 0.5]
### 준비 완료!! ###


## Finish